# Chapter-7
## Hypothesis & Inference
### Flipping a coin

$H_{0}$ = coin is fair. $p=0.5$.   
$H_{1}$ = coin is fair. $p\neq0.5$



In [3]:
from typing import Tuple
import math

def normal_approximation_to_binomial(n: int, p: float) -> Tuple[float, float]:
  """Returns mu and sigma corresponding to a Binomial(n, p)"""

  mu = p*n
  sigma = math.sqrt(p*(1-p)*n)
  return mu, sigma

In [4]:
def normal_cdf(x: float, mu: float=0, sigma: float=1) -> float:
  return(1+math.erf((x-mu) / math.sqrt(2)/sigma))/2

In [5]:
# the normal cdf is the probability that the variable is below a threshold
normal_probability_below = normal_cdf

# Above the threshold if it's not below the threshold
def normal_probability_above(lo: float, 
                             mu: float = 0,
                             sigma: float = 1) -> float:
  """The probability that an N(mu, sigma) is greater than lo."""
  return 1 - normal_cdf(lo, mu, sigma)

In [6]:
def normal_probability_between(lo: float,
                              hi: float,
                              mu: float = 0,
                              sigma: float=1):
  """The probability that an N(mu, sigma) is b/w lo and hi"""
  return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

def normal_probability_outside(lo: float, hi: float, mu: float, sigma: float=1)-> float:
  """The probability that an N(mu, sigma) is not between lo and hi"""
  return 1 - normal_probability_between(lo, hi, mu, sigma)

In [7]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)

In [8]:
mu_0, sigma_0

(500.0, 15.811388300841896)

In [9]:
def inverse_normal_cdf(p: float, 
                       mu: float = 0, 
                       sigma: float=1, 
                       tolerance: float = 0.00001) -> float:
                       """Find approximate inverse using binary search"""

                       # if not standard, computing standard and rescale

                       if mu!=0 or sigma!=1:
                         return mu + sigma + inverse_normal_cdf(p, tolerance=tolerance)
                       low_z = -10.0  # normal_cdf(-10) is (very close to) 0
                       hi_z = 10.0    # normal_cdf(10) is (very close to) 1
                       while hi_z - low_z > tolerance:
                         mid_z = (low_z + hi_z)/2
                         mid_p = normal_cdf(mid_z)
                         if mid_p < p:
                           low_z=mid_z
                         else:
                           hi_z=mid_z
                       return mid_z

In [10]:
def normal_upper_bound(probability: float, 
                       mu: float = 0, 
                       sigma: float = 1) -> float:
                       """Returns the z for which P(Z<=z) = probability"""
                       return inverse_normal_cdf(probability, mu, sigma)

In [11]:
def normal_lower_bound(probability: float,
                       mu: float = 0,
                       sigma: float = 1) -> float:
                       """Returns the z for which P(Z>=z)=probability"""
                       return inverse_normal_cdf(1-probability, mu, sigma)

In [12]:
def normal_two_sided_bounds(probability: float,
                            mu: float = 0,
                            sigma: float = 1) -> Tuple[float, float]:
                            """
                            Returns the symmetric (about the mean) bounds that contain the specified probability.
                            """
                            
                            tail_probability = (1 - probability)/2

                            # upper bound should have tail_probability above it.
                            upper_bound = normal_lower_bound(tail_probability, mu, sigma)

                            # lower bound should have tail_probability below it.
                            lower_bound = normal_upper_bound(tail_probability, mu, sigma)
                            
                            return lower_bound, upper_bound

In [13]:
lower_bound, upper_bound = normal_two_sided_bounds(0.95, mu_0, sigma_0)
lower_bound, upper_bound

(513.8514254559933, 517.7713511456906)

> `power` of a test: _probability of not making a Type-2 error_

In [14]:
# calculating the power of the test

lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)

# actual mu and sigma based on p=0.55
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)

# a type 2 error means we fail to reject the null hypothesis
# which will happen when X is still in our origianlinterval
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability
power

0.9905366515678999

In [15]:
hi = normal_upper_bound(0.95, mu_0, sigma_0)
hi

517.4562380780636

In [16]:
type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability
power

0.9807089934138811

### p-values

In [17]:
def two_sided_p_value(x: float, mu: float=0, sigma: float=1) -> float:
  """
  How likely are we to see a value at least as extreme as x (in either direction) if our values are from an N(mu, sigma)?
  """

  if x >= mu:
    # x is greater than the mean, so the tail is everything greater than x
    return 2 * normal_probability_above(x, mu, sigma)
  else:
    # x is less than the mean, so the tail is everything less than x
    return 2 * normal_probability_below(x, mu, sigma)

In [18]:
two_sided_p_value(529.5, mu_0, sigma_0)

0.06207721579598835

> a simulation to check the coin-toss

In [19]:
import random

extreme_value_count = 0

for _ in range(1000):
  num_heads = sum(1 if random.random()<0.5 else 0 for _ in range(1000))

  if num_heads >= 530 or num_heads <= 470:
    extreme_value_count +=1

In [20]:
extreme_value_count

76

_59 values out of 1000_

> 0.059 > 0.05. $H_{0}$ is not rejected

In [21]:
# for some number of heads > 500 like 542, the p-value would be:

two_sided_p_value(541.5, mu_0, sigma_0)

0.00867277726589677

### Confidence interval

In [22]:
p_hat = 540/1000
mu=p_hat
sigma = math.sqrt(p_hat * (1-p_hat)/1000)
normal_two_sided_bounds(0.95, mu, sigma)

(-1.4042021342047564, 2.5157235554925093)

### p-hacking

In [23]:
from collections import Counter
Counter([random.random()<0.5 for _ in range(1000)])

Counter({False: 458, True: 542})

In [24]:
from typing import List

def run_experiment() -> List[bool]:
  """Flips a fair coin 1000 times, True = heads, False = tails"""
  return [random.random()<0.5 for _ in range(1000)]

def reject_fairness(experiment: List[bool]) -> bool:
  """Using the 5% significance levels"""
  num_heads= len([flip for flip in experiment if flip])
  return num_heads < 469 or num_heads > 531

In [25]:
random.seed(0)
experiments = [run_experiment() for _ in range(1000)]

In [26]:
num_rejections = len([experiment for experiment in experiments if reject_fairness(experiment)])

In [27]:
assert num_rejections == 46

### A/B test

> As per the text mentioned:
- $N_{A}$: number of people seeing an Ad.
- $n_{A}$: number of people clicking on the Ad.
- Assume, each ad as a bernoulli trial.
- $p_{A}$: probability that someone clicks the ad.
- $n_{A}/N_{A}$: normal random variable with mean $p_{A}$ and standard deviation $\sigma_{A} = \sqrt{p_{A}(1-p_{A})/N_{A}}$
- Similarly for B.

In [28]:
# in coding terms
def estimated_parameters(N: int, n: int) -> Tuple[float, float]:
  p = n/N
  sigma = math.sqrt(p * (1-p)/N)
  return p, sigma

Assuming that $n_{A}/N_{A}$ and $n_{B}/N_{B}$ are independent.
> Individual bernoulli trials are.

Their difference will be (normal) with
1. _mean_ : $p_{B}-p_{A}$
2. standard deviation: $\sqrt{\sigma_{A}^2 + \sigma_{B}^2}$

In [29]:
# now testing the null hypothesis that pA and pB are same

def a_b_test_statistic(N_A: int, n_A: int, N_B: int, n_B: int) -> float:
  p_A, sigma_A = estimated_parameters(N_A, n_A)
  p_B, sigma_B = estimated_parameters(N_B, n_B)

  return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

# assumption: this should be approximately standard normal

In [30]:
# let's say we got nA = 200 out of 1000
# nB = 180 out of 1000

z = a_b_test_statistic(1000, 200, 1000, 180)
z

-1.1403464899034472

In [31]:
# probability of seeing such a large difference if the means were actually equal would be
two_sided_p_value(z)

0.2541419765422359

> no conclusion that there is much difference

In [32]:
# assume nB = 150 out of 1000, then
z = a_b_test_statistic(1000, 200, 1000, 150)
z, two_sided_p_value(z)

(-2.948839123097944, 0.003189699706216853)

> interesting.    
It means that there is only 0.003 probability that the difference will be this much if both ads were equally effective.

### Bayesian Inference

- It is an alternative approach which is about treating the unknown parameters as random variables.
- It involves choosing a prior to get an updated value for the posterior probability.


# Script Complete